In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Make Directed Graph
G = nx.DiGraph()

In [4]:
# Add Nodes
feat_size = 3
A = np.array([3, 1, 2])
B = np.array([5, 3, 1])
C = np.array([2, 3, 1])
D = np.array([5, 7, 2])
E = np.array([6, 4, 6])
G.add_node('A', v=A)
G.add_node('B', v=B)
G.add_node('C', v=C)
G.add_node('D', v=D)
G.add_node('E', v=E)

In [5]:
G.add_edges_from([('A', 'D'), ('D', 'E'), ('B', 'A'), ('C', 'B'), ('E', 'D'), ('C', 'E'), ('E', 'A')])

In [6]:
nx.draw(G, with_labels=True, font_weight='bold')

In [12]:
hidden_size = feat_size
# Define message, aggregate, update and output functions (one each), appied in order to each node
# Message: Takes an array (N_in, feats) (in-nodes) and applies the func
mF = lambda X: X * 2.  # Output: x.shape
# Aggregate: Takes a array of func (N_in x length) (in-node messages) and aggregates which in this case is average
aF = lambda X: np.mean(X, axis=0)  # Output: (N_in,)
# Update: Takes an array (2, feat) or (1, feat) which is the curr node feats and the aggregate
uF = lambda X: np.sum(X, axis=0)  # Output: (feat,)
# Output: Takes a vector (update for node) and applies func
oF = lambda x: x + 2.  # Output: x.shape

SyntaxError: invalid syntax (<ipython-input-12-f19c40fb7c61>, line 4)

In [9]:
# Propogate: (Ignore input) goes through each node and applies the 4 funcs above, at same time to each node
# so don't update the graph that is being propogated
new_node_dict = {}
outputs = []
for node, data in G.nodes(data=True):
    curr_feat = data['v']
    # Get predecessors
    preds_feats = np.array([G.nodes[n]['v'] for n in G.predecessors(node)])
    # Apply message, aggregate, update and output functions
    if preds_feats.size != 0:  # Has preds
        messages = mF(preds_feats)
        aggregate = aF(messages)
        stack = np.stack((curr_feat, aggregate))
        update = uF(stack)
    else:
        update = uF(curr_feat.reshape(1, -1))
    output = oF(update)
    outputs.append(output)
    new_node_dict[node] = update

In [10]:
new_node_dict

{'A': array([14.,  8.,  9.]),
 'B': array([9., 9., 3.]),
 'C': array([2, 3, 1]),
 'D': array([14., 12., 10.]),
 'E': array([13., 14.,  9.])}

In [11]:
outputs

[array([16., 10., 11.]),
 array([11., 11.,  5.]),
 array([4., 5., 3.]),
 array([16., 14., 12.]),
 array([15., 16., 11.])]

In [117]:
# Update graph (or make new in this case)
G_2 = G.copy()
nx.set_node_attributes(G_2, new_node_dict, 'v')